In [1]:
import numpy as np
import pandas as pd

In [2]:
train=pd.read_csv('IMDB/labeledTrainData.tsv', sep='\t',quoting=3,header=0)
test=pd.read_csv('IMDB/testData.tsv', sep='\t',quoting=3,header=0)
unlab_train=pd.read_csv('IMDB/unlabeledTrainData.tsv', sep='\t',quoting=3,header=0)
print("Read %d Labelled train reviews,Read %d test reviews and Read %d unLabelled train reviews\n" % (train['review'].size,test['review'].size,
     unlab_train['review'].size))

Read 25000 Labelled train reviews,Read 25000 test reviews and Read 50000 unLabelled train reviews



In [3]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

In [4]:
def review_to_words(review, remove_stopwords=False):
    review_text=BeautifulSoup(review).get_text()
    review_text=re.sub("[^a-zA-Z]", " ", review_text)
    words=review_text.lower().split()
    #Optionally remove stop words (false by default)
    if remove_stopwords:
        stops =set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return(words)

In [6]:
import nltk.data
tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')

In [7]:
def review_to_sentences(review,tokenizer,remove_stopwords =False):
    raw_sentences = tokenizer.tokenize(review.strip())   #Use the NLTK tokenizer to split the paragraph into sentences
    sentences =[]
    for raw_sentences in raw_sentences:
        if len(raw_sentences)>0:
            sentences.append(review_to_words(raw_sentences,remove_stopwords))
    return sentences

In [8]:
#Time taking events
sentences=[]
print("Parsing sentences from training sets")
for review in train['review']:
    sentences+=review_to_sentences(review,tokenizer)
    
    
print("Parsing sentences from unlabelled training sets")
for review in unlab_train['review']:
    sentences+=review_to_sentences(review,tokenizer)

Parsing sentences from training sets


C:\Users\VIPL013\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\VIPL013\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
C:\Users\VIPL013\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\VIPL013\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'...'" looks like a filename, not markup. You should probably op

Parsing sentences from unlabelled training sets


C:\Users\VIPL013\Anaconda3\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\VIPL013\Anaconda3\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\VIPL013\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'... ...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\VI

In [9]:
print(len(sentences))
print(sentences[0])

795538
['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']


In [17]:
#Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging

logging.basicConfig(format='%(asctime)s: %(levelname)s : %(message)s', level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

In [27]:
#!pip3 install gensim  installing gensim

C:\Users\VIPL013\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2018-08-28 08:53:28,879: INFO : 'pattern' package not found; tag filters are not available for English


In [32]:
from gensim.models import word2vec
print("Training model...")
model= word2vec.Word2Vec(sentences, workers=num_workers,
                        size=num_features,
                        min_count=min_word_count,
                        window=context,
                        sample=downsampling)
model.init_sims(replace=True)      # init_sims will make the model much more memory-efficient.
model_name="300features_40minwords_10context"
model.save(model_name) #save the model for later use. You can load it later using Word2Vec.load()

2018-08-28 08:58:49,424: INFO : collecting all words and their counts
2018-08-28 08:58:49,426: INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-08-28 08:58:49,544: INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types


Training model...


2018-08-28 08:58:49,650: INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2018-08-28 08:58:49,750: INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types
2018-08-28 08:58:49,864: INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2018-08-28 08:58:49,968: INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2018-08-28 08:58:50,084: INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2018-08-28 08:58:50,201: INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2018-08-28 08:58:50,319: INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2018-08-28 08:58:50,460: INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2018-08-28 08:58:50,577: INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word 

2018-08-28 08:58:57,479: INFO : PROGRESS: at sentence #750000, processed 16771230 words, keeping 120295 word types
2018-08-28 08:58:57,590: INFO : PROGRESS: at sentence #760000, processed 16990622 words, keeping 120930 word types
2018-08-28 08:58:57,687: INFO : PROGRESS: at sentence #770000, processed 17217759 words, keeping 121703 word types
2018-08-28 08:58:57,766: INFO : PROGRESS: at sentence #780000, processed 17447905 words, keeping 122402 word types
2018-08-28 08:58:57,868: INFO : PROGRESS: at sentence #790000, processed 17674981 words, keeping 123066 word types
2018-08-28 08:58:57,935: INFO : collected 123504 word types from a corpus of 17798082 raw words and 795538 sentences
2018-08-28 08:58:57,937: INFO : Loading a fresh vocabulary
2018-08-28 08:58:58,064: INFO : effective_min_count=40 retains 16490 unique words (13% of original 123504, drops 107014)
2018-08-28 08:58:58,066: INFO : effective_min_count=40 leaves 17238940 word corpus (96% of original 17798082, drops 559142)
2018

2018-08-28 08:59:52,307: INFO : EPOCH 3 - PROGRESS: at 62.82% examples, 611332 words/s, in_qsize 7, out_qsize 0
2018-08-28 08:59:53,314: INFO : EPOCH 3 - PROGRESS: at 67.69% examples, 611746 words/s, in_qsize 8, out_qsize 0
2018-08-28 08:59:54,328: INFO : EPOCH 3 - PROGRESS: at 72.66% examples, 612835 words/s, in_qsize 8, out_qsize 1
2018-08-28 08:59:55,337: INFO : EPOCH 3 - PROGRESS: at 77.66% examples, 613974 words/s, in_qsize 7, out_qsize 0
2018-08-28 08:59:56,347: INFO : EPOCH 3 - PROGRESS: at 82.61% examples, 614496 words/s, in_qsize 8, out_qsize 0
2018-08-28 08:59:57,348: INFO : EPOCH 3 - PROGRESS: at 87.31% examples, 613657 words/s, in_qsize 8, out_qsize 1
2018-08-28 08:59:58,358: INFO : EPOCH 3 - PROGRESS: at 92.04% examples, 613036 words/s, in_qsize 8, out_qsize 0
2018-08-28 08:59:59,359: INFO : EPOCH 3 - PROGRESS: at 96.86% examples, 612740 words/s, in_qsize 7, out_qsize 0
2018-08-28 08:59:59,975: INFO : worker thread finished; awaiting finish of 3 more threads
2018-08-28 08:

In [53]:
model.doesnt_match("man  woman child kitchen ".split())

C:\Users\VIPL013\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'kitchen'

In [57]:
model.doesnt_match('paris berlin london India'.split()) #not correct

C:\Users\VIPL013\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
2018-08-28 09:19:38,304: WARNING : vectors for words {'India'} are not present in the model, ignoring these words


'london'

In [55]:
model.doesnt_match('france england germany berlin'.split())

C:\Users\VIPL013\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'berlin'

In [58]:
model.most_similar('man')

C:\Users\VIPL013\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('woman', 0.6322795152664185),
 ('lady', 0.6022685766220093),
 ('lad', 0.5814216136932373),
 ('farmer', 0.5361886024475098),
 ('soldier', 0.5300039052963257),
 ('men', 0.5290796160697937),
 ('millionaire', 0.5165280103683472),
 ('businessman', 0.5111342668533325),
 ('person', 0.5093293786048889),
 ('policeman', 0.507225751876831)]

In [59]:
model.most_similar('queen')

C:\Users\VIPL013\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('princess', 0.6921592950820923),
 ('belle', 0.6261844635009766),
 ('bride', 0.6246557235717773),
 ('victoria', 0.6203603744506836),
 ('maid', 0.6162511110305786),
 ('dame', 0.5923529267311096),
 ('stepmother', 0.5910489559173584),
 ('widow', 0.586331844329834),
 ('marlene', 0.582047700881958),
 ('countess', 0.5790597796440125)]

In [60]:
model.most_similar('awful')

C:\Users\VIPL013\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('terrible', 0.7781771421432495),
 ('atrocious', 0.7508953809738159),
 ('horrible', 0.7311934232711792),
 ('abysmal', 0.7290616035461426),
 ('dreadful', 0.7164966464042664),
 ('horrendous', 0.6882622241973877),
 ('horrid', 0.6806074380874634),
 ('appalling', 0.6685008406639099),
 ('lousy', 0.6501781344413757),
 ('laughable', 0.6276453733444214)]

In [70]:
model.most_similar('bad')

C:\Users\VIPL013\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('terrible', 0.6780447363853455),
 ('horrible', 0.6577404141426086),
 ('good', 0.6260221600532532),
 ('crappy', 0.6259921789169312),
 ('lousy', 0.6008751392364502),
 ('awful', 0.59053635597229),
 ('stupid', 0.5894876718521118),
 ('cheesy', 0.5790930986404419),
 ('lame', 0.5670125484466553),
 ('atrocious', 0.544480562210083)]

In [81]:
model.doesnt_match(' inferior bad good'.split())

C:\Users\VIPL013\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'inferior'